In [56]:
#Imports
import random
import numpy as np
import pandas as pd

import trax
from trax import layers as tl
from trax.fastmath import numpy as fastnp
from trax.supervised import training

In [ ]:
#Creating generator functions with training and evaluation data using trax
train_stream_fn = trax.data.TFDS('opus/medical',
                                 data_dir='./data/',
                                 keys=('en', 'de'),
                                 eval_holdout_size=0.01, # 1% for eval
                                 train=True
                                )

eval_stream_fn = trax.data.TFDS('opus/medical',
                                data_dir='./data/',
                                keys=('en', 'de'),
                                eval_holdout_size=0.01, # 1% for eval                                
                                train=False
                               )

In [65]:
train_stream = train_stream_fn()
eval_stream = eval_stream_fn()

In [66]:
#Tokenizing by subwords
data_dir = './data/'
vocab_file = 'ende_32k.subword'
tokenized_train_stream = trax.data.Tokenize(vocab_file=vocab_file, vocab_dir=data_dir)(train_stream)
tokenized_eval_stream = trax.data.Tokenize(vocab_file=vocab_file, vocab_dir=data_dir)(eval_stream)

In [67]:
#Adding <End of sentence> token to each tokenized sentence
EOS =  1
def append_eos(stream):
    for (inputs, targets) in stream:
        inputs_with_eos = list(inputs) + [EOS]
        targets_with_eos = list(targets) + [EOS]
        yield np.array(inputs_with_eos), np.array(targets_with_eos)

# append EOS to the train data
tokenized_train_stream = append_eos(tokenized_train_stream)

# append EOS to the eval data
tokenized_eval_stream = append_eos(tokenized_eval_stream)    

In [68]:
# Setup helper functions for tokenizing and detokenizing sentences

def tokenize(input_str, vocab_file=None, vocab_dir=None):
    """Encodes a string to an array of integers

    Args:
        input_str (str): human-readable string to encode
        vocab_file (str): filename of the vocabulary text file
        vocab_dir (str): path to the vocabulary file
  
    Returns:
        numpy.ndarray: tokenized version of the input string
    """
    
    # Set the encoding of the "end of sentence" as 1
    EOS = 1
    
    # Use the trax.data.tokenize method. It takes streams and returns streams,
    # we get around it by making a 1-element stream with `iter`.
    inputs =  next(trax.data.tokenize(iter([input_str]),
                                      vocab_file=vocab_file, vocab_dir=vocab_dir))
    
    # Mark the end of the sentence with EOS
    inputs = list(inputs) + [EOS]
    
    # Adding the batch dimension to the front of the shape
    batch_inputs = np.reshape(np.array(inputs), [1, -1])
    
    return batch_inputs


def detokenize(integers, vocab_file=None, vocab_dir=None):
    """Decodes an array of integers to a human readable string

    Args:
        integers (numpy.ndarray): array of integers to decode
        vocab_file (str): filename of the vocabulary text file
        vocab_dir (str): path to the vocabulary file
  
    Returns:
        str: the decoded sentence.
    """
    
    # Remove the dimensions of size 1
    integers = list(np.squeeze(integers))
    
    # Set the encoding of the "end of sentence" as 1
    EOS = 1
    
    # Remove the EOS to decode only the original tokens
    if EOS in integers:
        integers = integers[:integers.index(EOS)] 
    
    return trax.data.detokenize(integers, vocab_file=vocab_file, vocab_dir=vocab_dir)

In [69]:
#Truncating excesive lenght sentences
filtered_train_stream = trax.data.FilterByLength(
    max_length=512, length_keys=[0, 1])(tokenized_train_stream)
filtered_eval_stream = trax.data.FilterByLength(
    max_length=512, length_keys=[0, 1])(tokenized_eval_stream)

In [70]:
#Bucketing by length with Trax
boundaries =  [8,   16,  32, 64, 128, 256, 512] 
batch_sizes = [256, 128, 64, 32, 16,    8,   4,  2]

train_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes,
    length_keys=[0, 1] 
)(filtered_train_stream)

eval_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes,
    length_keys=[0, 1]  
)(filtered_eval_stream)

In [71]:
# Add masking for the padding .
train_batch_stream = trax.data.AddLossWeights(id_to_mask=0)(train_batch_stream)
eval_batch_stream = trax.data.AddLossWeights(id_to_mask=0)(eval_batch_stream)

In [72]:
#Setting encoder layers
def input_encoder_fn(input_vocab_size, d_model, n_encoder_layers):
    """ Input encoder runs on the input sentence and creates
    activations that will be the keys and values for attention.
    
    Args:
        input_vocab_size: int: vocab size of the input
        d_model: int:  depth of embedding (n_units in the LSTM cell)
        n_encoder_layers: int: number of LSTM layers in the encoder
    Returns:
        tl.Serial: The input encoder
    """
    input_encoder = tl.Serial( 
        # create an embedding layer to convert tokens to vectors
        tl.Embedding(vocab_size=input_vocab_size, d_feature= d_model),

        # feed the embeddings to the LSTM layers. 
        [tl.LSTM(n_units = d_model) for l in range(n_encoder_layers)]
    )
    return input_encoder

In [73]:
#Setting pre attention decoder architecture
#Note: Simple way to get the hidden states for the attention mechanism implementation later.
def pre_attention_decoder_fn(mode, target_vocab_size, d_model):
    """ Pre-attention decoder runs on the targets and creates
    activations that are used as queries in attention.
    
    Args:
        mode: str: 'train' or 'eval'
        target_vocab_size: int: vocab size of the target
        d_model: int:  depth of embedding (n_units in the LSTM cell)
    Returns:
        tl.Serial: The pre-attention decoder
    """
    
    # create a serial network
    pre_attention_decoder = tl.Serial(
        # shift right to insert start-of-sentence token and implement
        # teacher forcing during training
        tl.ShiftRight(mode=mode),
        # run an embedding layer to convert tokens to vectors
        tl.Embedding(vocab_size=target_vocab_size,d_feature=d_model),
        # feed to an LSTM layer
        tl.LSTM(d_model)
    ) 
    return pre_attention_decoder

In [74]:
#Function to prepare attention layer input with queries,keys, values and attentionmask
def prepare_attention_input(encoder_activations, decoder_activations, inputs):
    """Prepare queries, keys, values and mask for attention.
    
    Args:
        encoder_activations fastnp.array(batch_size, padded_input_length, d_model): output from the input encoder
        decoder_activations fastnp.array(batch_size, padded_input_length, d_model): output from the pre-attention decoder
        inputs fastnp.array(batch_size, padded_input_length): input tokens
    
    Returns:
        queries, keys, values and mask for attention.
    """

    keys = encoder_activations
    values = encoder_activations
    queries = decoder_activations

    mask = inputs != 0
    mask = fastnp.reshape(mask, (mask.shape[0], 1, 1, mask.shape[1]))
    mask = mask + fastnp.zeros((1, 1, decoder_activations.shape[1], 1))
    
    return queries, keys, values, mask

In [75]:
#Setting Attention layers
def AttentionQKV(d_feature, n_heads=1, dropout=0.0, mode='train'):
  """Returns a layer that maps (q, k, v, mask) to (activations, mask).

  See `Attention` above for further context/details.

  Args:
    d_feature: Depth/dimensionality of feature embedding.
    n_heads: Number of attention heads.
    dropout: Probababilistic rate for internal dropout applied to attention
        activations (based on query-key pairs) before dotting them with values.
    mode: Either 'train' or 'eval'.
  """
  return cb.Serial(
      cb.Parallel(
          core.Dense(d_feature),
          core.Dense(d_feature),
          core.Dense(d_feature),
      ),
      PureAttention(  # pylint: disable=no-value-for-parameter
          n_heads=n_heads, dropout=dropout, mode=mode),
      core.Dense(d_feature),
  )

In [76]:
#Build final model adding all modules created above
def NMTAttn(input_vocab_size=33300,
            target_vocab_size=33300,
            d_model=1024,
            n_encoder_layers=2,
            n_decoder_layers=2,
            n_attention_heads=4,
            attention_dropout=0.0,
            mode='train'):
    """Returns an LSTM sequence-to-sequence model with attention.

    The input to the model is a pair (input tokens, target tokens), e.g.,
    an English sentence (tokenized) and its translation into German (tokenized).

    Args:
    input_vocab_size: int: vocab size of the input
    target_vocab_size: int: vocab size of the target
    d_model: int:  depth of embedding (n_units in the LSTM cell)
    n_encoder_layers: int: number of LSTM layers in the encoder
    n_decoder_layers: int: number of LSTM layers in the decoder after attention
    n_attention_heads: int: number of attention heads
    attention_dropout: float, dropout for the attention layer
    mode: str: 'train', 'eval' or 'predict', predict mode is for fast inference

    Returns:
    An LSTM sequence-to-sequence model with attention.
    """

    input_encoder = input_encoder_fn(input_vocab_size,d_model,n_encoder_layers)
    pre_attention_decoder = pre_attention_decoder_fn(mode,target_vocab_size,d_model)

    # create a serial network
    model = tl.Serial( 
        
    # copy input tokens and target tokens as they will be needed later.
    tl.Select([0,1,0,1]),
      
    # run input encoder on the input and pre-attention decoder the target.
    tl.Parallel(input_encoder, pre_attention_decoder),
      
    # prepare queries, keys, values and mask for attention.
    tl.Fn('PrepareAttentionInput', f=prepare_attention_input, n_out=4),
      
    # run the AttentionQKV layer
    # nest it inside a Residual layer to add to the pre-attention decoder activations(i.e. queries)
    tl.Residual(tl.AttentionQKV(d_model, n_heads=n_attention_heads, dropout=attention_dropout, mode=mode)),
    
    # drop attention mask (i.e. index = None
    tl.Select([0,2]),
      
    # run the rest of the RNN decoder
    [tl.LSTM(n_units=d_model) for _ in range(n_decoder_layers)],
      
    # prepare output by making it the right size
    tl.Dense(n_units=target_vocab_size),
      
    # Transforming output from dense with Log-softmax 
    tl.LogSoftmax()
    )
    
    return model

In [77]:
model = NMTAttn()
print(model)

Serial_in2_out2[
  Select[0,1,0,1]_in2_out4
  Parallel_in2_out2[
    Serial[
      Embedding_33300_1024
      LSTM_1024
      LSTM_1024
    ]
    Serial[
      Serial[
        ShiftRight(1)
      ]
      Embedding_33300_1024
      LSTM_1024
    ]
  ]
  PrepareAttentionInput_in3_out4
  Serial_in4_out2[
    Branch_in4_out3[
      None
      Serial_in4_out2[
        _in4_out4
        Serial_in4_out2[
          Parallel_in3_out3[
            Dense_1024
            Dense_1024
            Dense_1024
          ]
          PureAttention_in4_out2
          Dense_1024
        ]
        _in2_out2
      ]
    ]
    Add_in2
  ]
  Select[0,2]_in3_out2
  LSTM_1024
  LSTM_1024
  Dense_33300
  LogSoftmax
]


In [78]:
#Setting training generator  with the parameters below
training_generator = training.TrainTask(
        labeled_data= train_batch_stream,
        loss_layer= tl.CrossEntropyLoss(), #loss function to optimize
        optimizer= trax.optimizers.Adam(0.01), #optimizer
        lr_schedule = trax.lr.warmup_and_rsqrt_decay(n_warmup_steps=1000, max_value=0.01), #learning rate schedule which follows sqrt decay
        n_steps_per_checkpoint= 10
    )

In [79]:
#Setting evaluation generator
eval_generator = training.EvalTask(
    labeled_data=eval_batch_stream,
    metrics=[tl.CrossEntropyLoss(), tl.Accuracy()], 
)

In [80]:
# Output directory
output_dir = 'output_dir/'

# remove old model if it exists. restarts training.
!rm -f ~/output_dir/model.pkl.gz  

# define the training loop
training_loop = training.Loop(NMTAttn(mode='train'),
                              training_generator,
                              eval_tasks=[eval_generator],
                              output_dir=output_dir)

In [ ]:
training_loop.run(10) #10 epochs training test

# Loading checkpoint

In [81]:
# instantiate the model built in eval mode
model = NMTAttn(mode='eval')

# initialize weights from a pre-trained model
model.init_from_file("model.pkl.gz", weights_only=True)
model = tl.Accelerate(model)

# Sampling and decoding

In [82]:
#Inference function for next symbol prediction using model previously trained
def next_symbol(NMTAttn, input_tokens, cur_output_tokens, temperature):
    """Returns the index of the next token.

    Args:
        NMTAttn (tl.Serial): An LSTM sequence-to-sequence model with attention.
        input_tokens (np.ndarray 1 x n_tokens): tokenized representation of the input sentence
        cur_output_tokens (list): tokenized representation of previously translated words
        temperature (float): parameter for sampling ranging from 0.0 to 1.0.
            0.0: same as argmax, always pick the most probable token
            1.0: sampling from the distribution (can sometimes say random things)

    Returns:
        int: index of the next token in the translated sentence
        float: log probability of the next symbol
    """
    # set the length of the current output tokens
    token_length = len(cur_output_tokens)
    
    # calculate next power of 2 for padding length 
    padded_length = np.power(2,int(np.ceil(np.log2(token_length +1))))

    # pad cur_output_tokens up to the padded_length
    padded = cur_output_tokens + list(np.zeros(padded_length-token_length,dtype=int))
    
    # model expects the output to have an axis for the batch size in front so
    # convert `padded` list to a numpy array with shape (1, <padded_length>)
    padded_with_batch = np.expand_dims(padded, axis=0)

    # get the model prediction
    output, _ = NMTAttn((input_tokens, padded_with_batch))
    
    # get log probabilities from the last token output
    log_probs = output[0,token_length,:]

    # get the next symbol by getting a logsoftmax sample 
    symbol = int(tl.logsoftmax_sample(log_probs, temperature)) 
    return symbol, float(log_probs[symbol])

In [83]:
# Calling function next symbol until EOS token is selected. Temperature will condition the randomness of the sampling, selecting randomly one of the possible tokens depeding on their log probs in the probability distribution.
# Temperature = 0 means no random sampling and greedy decoding will occur(the token with highest log prob will be selected as next symbol)
# With high temperatures tokens with low log probs will have higher probabilities of being selected in the random sampling
def sampling_decode(input_sentence, NMTAttn = None, temperature=0.0, vocab_file=None, vocab_dir=None, next_symbol=next_symbol, tokenize=tokenize, detokenize=detokenize):
    """Returns the translated sentence.

    Args:
        input_sentence (str): sentence to translate.
        NMTAttn (tl.Serial): An LSTM sequence-to-sequence model with attention.
        temperature (float): parameter for sampling ranging from 0.0 to 1.0.
            0.0: same as argmax, always pick the most probable token
            1.0: sampling from the distribution (can sometimes say random things)
        vocab_file (str): filename of the vocabulary
        vocab_dir (str): path to the vocabulary file

    Returns:
        tuple: (list, str, float)
            list of int: tokenized version of the translated sentence
            float: log probability of the translated sentence
            str: the translated sentence
    """

    # encode the input sentence
    input_tokens = tokenize(input_sentence,vocab_file,vocab_dir)
    
    cur_output_tokens = []
    cur_output = 0
    # Set the encoding of the "end of sentence" as 1
    EOS = 1
    
    # check that the current output is not the end of sentence token
    while cur_output != EOS:
        
        # update the current output token by getting the index of the next word 
        cur_output, log_prob = next_symbol(NMTAttn, input_tokens, cur_output_tokens, temperature)
        
        # append the current output token to the list of output tokens
        cur_output_tokens.append(cur_output)  
    
    # detokenize the output tokens
    sentence = detokenize(cur_output_tokens, vocab_file, vocab_dir)

    return cur_output_tokens, log_prob, sentence

In [84]:
# Minimum Bayes Risk decoding will be carried.
# First  a set of random samples will be generated
# Then, they will be scored with ROUGE score with each other
# Finally the best candidate through MBR decoding will be the one with highest score

In [85]:
# Funtion to generate several random samples
def generate_samples(sentence, n_samples, NMTAttn=None, temperature=0.6, vocab_file=None, vocab_dir=None, sampling_decode=sampling_decode, next_symbol=next_symbol, tokenize=tokenize, detokenize=detokenize):
    """Generates samples using sampling_decode()

    Args:
        sentence (str): sentence to translate.
        n_samples (int): number of samples to generate
        NMTAttn (tl.Serial): An LSTM sequence-to-sequence model with attention.
        temperature (float): parameter for sampling ranging from 0.0 to 1.0.
            0.0: same as argmax, always pick the most probable token
            1.0: sampling from the distribution (can sometimes say random things)
        vocab_file (str): filename of the vocabulary
        vocab_dir (str): path to the vocabulary file
        
    Returns:
        tuple: (list, list)
            list of lists: token list per sample
            list of floats: log probability per sample
    """
    # define lists to contain samples and probabilities
    samples, log_probs = [], []

    # run a for loop to generate n samples
    for _ in range(n_samples):
        
        # get a sample using the sampling_decode() function
        sample, logp, _ = sampling_decode(sentence, NMTAttn, temperature, vocab_file=vocab_file, vocab_dir=vocab_dir, next_symbol=next_symbol)
        
        # append the token list to the samples list
        samples.append(sample)
        
        # append the log probability to the log_probs list
        log_probs.append(logp)
                
    return samples, log_probs

In [86]:
#Function to calculate rouge score between pairs.
def rouge1_similarity(system, reference):
    """Returns the ROUGE-1 score between two token lists

    Args:
        system (list of int): tokenized version of the system translation
        reference (list of int): tokenized version of the reference translation

    Returns:
        float: overlap between the two token lists
    """    

    # make a frequency table of the system tokens 
    sys_counter = Counter(system)
    
    # make a frequency table of the reference tokens 
    ref_counter = Counter(reference)
    
    # initialize overlap to 0
    overlap = 0
    
    # run a for loop over the sys_counter object 
    for token in sys_counter:
        
        # lookup the value of the token in the sys_counter dictionary 
        token_count_sys = sys_counter.get(token,0)
        
        # lookup the value of the token in the ref_counter dictionary 
        token_count_ref = ref_counter.get(token,0)
        
        # update the overlap by getting the smaller number between the two token counts above
        overlap += min(token_count_sys, token_count_ref)
    
    # get the precision
    precision = overlap/len(system)
    
    # get the recall
    recall = overlap/len(reference)
    
    if precision + recall != 0: 
        # compute the f1-score
        rouge1_score = 2 * ((precision * recall)/(precision + recall))
    else:
        rouge1_score = 0 
    
    return rouge1_score

In [87]:
#Calculates ROUGE score arithmetic mean for each sample.
def average_overlap(similarity_fn, samples, *ignore_params):
    """Returns the arithmetic mean of each candidate sentence in the samples

    Args:
        similarity_fn (function): similarity function used to compute the overlap
        samples (list of lists): tokenized version of the translated sentences
        *ignore_params: additional parameters will be ignored

    Returns:
        dict: scores of each sample
            key: index of the sample
            value: score of the sample
    """  
    
    # initialize dictionary
    scores = {}
    
    # run a for loop for each sample
    for index_candidate, candidate in enumerate(samples):    
       
        # initialize overlap
        overlap = 0
        
        # run a for loop for each sample
        for index_sample, sample in enumerate(samples): 

            # skip if the candidate index is the same as the sample index
            if index_candidate == index_sample:
                continue
                
            # get the overlap between candidate and sample using the similarity function
            sample_overlap = rouge1_similarity(candidate, sample)
            
            # add the sample overlap to the total overlap
            overlap += sample_overlap
            
        # get the score for the candidate by computing the average
        score = overlap/(len(samples)-1)
        
        # save the score in the dictionary.
        scores[index_candidate] = score
    return scores

In [88]:
#Funtion to implement minimum bayes risk decoding using previous helper functions
def mbr_decode(sentence, n_samples, score_fn, similarity_fn, NMTAttn=None, temperature=0.6, vocab_file=None, vocab_dir=None, generate_samples=generate_samples, sampling_decode=sampling_decode, next_symbol=next_symbol, tokenize=tokenize, detokenize=detokenize):
    """Returns the translated sentence using Minimum Bayes Risk decoding

    Args:
        sentence (str): sentence to translate.
        n_samples (int): number of samples to generate
        score_fn (function): function that generates the score for each sample
        similarity_fn (function): function used to compute the overlap between a pair of samples
        NMTAttn (tl.Serial): An LSTM sequence-to-sequence model with attention.
        temperature (float): parameter for sampling ranging from 0.0 to 1.0.
            0.0: same as argmax, always pick the most probable token
            1.0: sampling from the distribution (can sometimes say random things)
        vocab_file (str): filename of the vocabulary
        vocab_dir (str): path to the vocabulary file

    Returns:
        str: the translated sentence
    """
    
    # generate samples
    samples, log_probs = generate_samples(sentence, n_samples, NMTAttn=NMTAttn, temperature=temperature, vocab_file=vocab_file, vocab_dir=vocab_dir)
    
    # use the scoring function to get a dictionary of scores
    scores = score_fn(similarity_fn, samples, log_probs)
    
    # find the key with the highest score
    max_score_key = max(scores, key=scores.get)
    
    # detokenize the token list associated with the max_score_key
    translated_sentence = detokenize(samples[max_score_key], vocab_file, vocab_dir)
    
    return (translated_sentence, max_score_key, scores)

# Inference with examples

In [89]:
examples = ['Robert was a good king.', 
            'He had a great army.',
            'He wanted to bring peace to his kingdom.',
            'There were many others who wanted to become king.',
            'They started plotting against him.',
            'Their plots were failing because of some trusted friends of the king.',
            'Then they started killing those trusted friends.',
            'Eventually, they succeeded in their plan of killing the king.',
            'Did they make a good move?',
            'Can they find a new king without dispute?',
            'After the death of the king, everyone wanted to be a king.',
            'A great chaos broke out in the kingdom.',
            'People were anxious and unhappy.',
            'War does not bring anything good to the common people.'
            'It only brings sorrow and disma']


In [90]:
#Greedy decoding over examples(temperature = 0)
greedy_decoded_sentences=[]
for example in examples:
    translation = mbr_decode(example, 4, average_overlap, rouge1_similarity, model, temperature= 0, vocab_file=vocab_file, vocab_dir=data_dir)[0]
    greedy_decoded_sentences.append(translation)

In [93]:
for sentence in greedy_decoded_sentences:
    print(sentence)

Robert war ein guter König.
Er hatte eine große Armee.
Er wollte seinem Reich Frieden bringen.
Es gab viele andere, die sich zum König machen wollten.
Sie begannen, gegen ihn zu protestieren.
Ihre Grundstücke sind durch einige treuhändige Freunde des Königs nicht zu übertreffen.
Dann begannen sie, diese vertrauenswürdigen Freunde zu töten.
Schließlich gelang es ihnen, den König zu töten.
Haben sie einen guten Schritt gemacht?
Können sie einen neuen König finden ohne Streit?
Nach dem Tod des Königs wünschte jeder König.
Im Reich ist ein großer Chaos ausbrachen.
Die Leute waren beunruhigt und unglücklich.
Krieg bringt nichts Gutes für das Volk.Es bringt nur Trauer und Elend.


In [96]:
#Random sampling with temperature = 0.6 over examples
t6_sentences=[]
for example in examples:
    translation = mbr_decode(example, 4, average_overlap, rouge1_similarity, model, temperature= 0.6, vocab_file=vocab_file, vocab_dir=data_dir)[0]
    t6_sentences.append(translation)

In [99]:
Translator_table = pd.DataFrame({'Examples':examples,'Translated sentences with Temperature=0':greedy_decoded_sentences,'Translated sentences with temperature 0.6':t6_sentences})

In [100]:
Translator_table

,Examples,Translated sentences with Temperature=0,Translated sentences with temperature 0.6
0,Robert was a good king.,Robert war ein guter König.,Robert war ein guter König.
1,He had a great army.,Er hatte eine große Armee.,Er hatte eine große Armee.
2,He wanted to bring peace to his kingdom.,Er wollte seinem Reich Frieden bringen.,Er wollte seinem Reich Frieden bringen.
3,There were many others who wanted to become king.,"Es gab viele andere, die sich zum König machen...","Es gab viele andere, die sich zum König machen..."
4,They started plotting against him.,"Sie begannen, gegen ihn zu protestieren.","Sie begannen, gegen ihn zu klagen."
5,Their plots were failing because of some trust...,Ihre Grundstücke sind durch einige treuhändige...,Ihre Grundstücke sind aufgrund gewisser Treuhä...
6,Then they started killing those trusted friends.,"Dann begannen sie, diese vertrauenswürdigen Fr...","Dann begannen sie, diese vertrauenswürdigen Fr..."
7,"Eventually, they succeeded in their plan of ki...","Schließlich gelang es ihnen, den König zu töten.","Schließlich gelang es ihnen ihren Plan, den Kö..."
8,Did they make a good move?,Haben sie einen guten Schritt gemacht?,Haben sie einen guten Schritt gemacht?
9,Can they find a new king without dispute?,Können sie einen neuen König finden ohne Streit?,"Können sie einen neuen König finden, ohne Streit?"
